ESQUELETO:

Este es el archivo para subir el modelo a la nube.


CÓMO FUNCIONA:

1. Tomar un modelo ya realizado.

2. Lo sube a la nube.

ASEGÚRENSE DE QUE SE COORDINAN LO NECESARIO PARA QUE LOS NOMBRES COINCIDAN (no es de chill cambiar el estándar xd).

In [1]:
## STEP 1: Set up the workspace.

import json

#GETTING MY ID:
my_id = config['id']

from azureml.core import Workspace

ws = Workspace.create(name="workspace",
                      subscription_id = my_id,
                      resource_group = "deployment",
                      location = "brazilsouth")

from azureml.core.model import Model

mname = "model"
registered_model = Model.register(model_path="model.pkl",
                                  model_name=mname,
                                  workspace=ws)

Cleaning up past default Resource Group Deployments on the subscription to avoid limit of 10
Deleting past Resource Group Deployment with name: DeployResourceGroup-5483a0926f
Deploying KeyVault with name workspackeyvaultf993b9e7.
Deploying StorageAccount with name workspacstorage3b0082d40.
Deployed KeyVault with name workspackeyvaultf993b9e7. Took 27.84 seconds.
Deploying AppInsights with name workspacinsights1dd1533e.
Deployed AppInsights with name workspacinsights1dd1533e. Took 63.0 seconds.
Deploying Workspace with name workspace.
Deployed Workspace with name workspace. Took 40.13 seconds.
Registering model model


In [11]:
################ score.py ###################

umb = open("umbral.json", "r")
umb = json.load(umb)
umbral = umb["umbral"][0]

scorepy = """
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('model')
  model = joblib.load(model_path)

def run(raw_data):
  try:
    data = json.loads(raw_data)['data']
    data = pd.DataFrame(data)
    
    # These are the 15 features selected in your model2.ipynb
    required_columns = [
        ' ROA(C) before interest and depreciation before interest',
        ' ROA(A) before interest and % after tax',
        ' ROA(B) before interest and depreciation after tax',
        ' Persistent EPS in the Last Four Seasons',
        ' Per Share Net profit before tax (Yuan ¥)',
        ' Debt ratio %',
        ' Net worth/Assets',
        ' Borrowing dependency',
        ' Net profit before tax/Paid-in capital',
        ' Working Capital to Total Assets',
        ' Current Liability to Assets',
        ' Retained Earnings to Total Assets',
        ' Current Liability to Current Assets',
        ' Net Income to Total Assets',
        ' Net Income to Stockholder\\'s Equity'
    ]
    
    # Check if all required columns are present
    for col in required_columns:
        if col not in data.columns:
            return json.dumps(f"Missing required column: {{col}}")
    
    # Select only the required columns in the right order
    data_features = data[required_columns]
    
    # Try with predict_proba first, fall back to predict if it fails
    try:
        # Try to use predict_proba if available
        result = model.predict_proba(data_features)[:, 1].tolist()
        umbral = {umbral}  # The threshold from your umbral.json
        result_finals = [1 if x > umbral else 0 for x in result]
    except (AttributeError, TypeError):
        # If predict_proba is not available, use predict directly
        result_finals = model.predict(data_features).tolist()
    
    return json.dumps(result_finals)
  except Exception as e:
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [12]:
## STEP 2: Inference configuration: the blueprints for the German carpenter about how to build your furniture.

from azureml.core.environment import Environment
virtual_env = Environment("env-4-housing")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas', 'scikit-learn', 'numpy', 'joblib']) ## Asegúrense de coordinarse con el departamento de modelos para incluir los paquetes correctos. Si no, BOOM!


from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) ## ASEGÚRENSE DE ASIGNAR SUFICIENTE MADERA PARA SUS MUEBLES.

service = Model.deploy(workspace=ws,
                       name='service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

/var/folders/b5/6d3jsq3n12j27vfn8yqh_y500000gn/T/ipykernel_9719/381904218.py:18: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [13]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()

ACI service creation operation finished, operation "Succeeded"
